In [1]:
#filedrive="/content/drive/My Drive/0Satellite_Images_Unsupervised/MNIST_PyTorch"
#from google.colab import drive
#drive.mount('/content/drive')
filedrive="data/0Satellite_Images_Unsupervised/MNIST_PyTorch"

import torch, torchvision
from torch import nn, optim
import torch.nn.functional as F

In [2]:
"""
class Encoder(nn.Module):
    def __init__(self, bands_nb, patch_size):
        input_size = (bands_nb, patch_size, patch_size)

        super(Encoder, self).__init__()

        # Stage 1
        # Feature extraction
        self.conv11 = nn.Conv2d(bands_nb, 32, kernel_size=3, padding=1)
        self.bn11 = nn.BatchNorm2d(32)
        self.conv12 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn12 = nn.BatchNorm2d(32)
        self.conv13 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn13 = nn.BatchNorm2d(64)
        self.conv14 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn14 = nn.BatchNorm2d(64)
        self.activation1 = nn.ReLU()
        self.activation2 = nn.Sigmoid()
        self.l2norm = L2()


    def forward(self, x):
        # Stage 1
        x11 = F.relu(self.bn11(self.conv11(x)))
        x12 = F.relu(self.bn12(self.conv12(x11)))
        x13 = F.relu(self.bn13(self.conv13(x12)))
        x14 = 
        size14 = x14.size()
        x14_ = x14.view(size14[0], size14[1], size14[2]*size14[3])
        x14_ = F.normalize(x14_, p=2, dim=2)
        encoded = x14_.view(size14[0], size14[1], size14[2], size14[3])


        return encoded
"""

'\nclass Encoder(nn.Module):\n    def __init__(self, bands_nb, patch_size):\n        input_size = (bands_nb, patch_size, patch_size)\n\n        super(Encoder, self).__init__()\n\n        # Stage 1\n        # Feature extraction\n        self.conv11 = nn.Conv2d(bands_nb, 32, kernel_size=3, padding=1)\n        self.bn11 = nn.BatchNorm2d(32)\n        self.conv12 = nn.Conv2d(32, 32, kernel_size=3, padding=1)\n        self.bn12 = nn.BatchNorm2d(32)\n        self.conv13 = nn.Conv2d(32, 64, kernel_size=3, padding=1)\n        self.bn13 = nn.BatchNorm2d(64)\n        self.conv14 = nn.Conv2d(64, 64, kernel_size=3, padding=1)\n        self.bn14 = nn.BatchNorm2d(64)\n        self.activation1 = nn.ReLU()\n        self.activation2 = nn.Sigmoid()\n        self.l2norm = L2()\n\n\n    def forward(self, x):\n        # Stage 1\n        x11 = F.relu(self.bn11(self.conv11(x)))\n        x12 = F.relu(self.bn12(self.conv12(x11)))\n        x13 = F.relu(self.bn13(self.conv13(x12)))\n        x14 = \n        size14

In [3]:
class Classifier(nn.Module):
    def __init__(self, bands_nb, patch_size):

        input_size = (bands_nb, patch_size, patch_size)
      
        super().__init__()
        self.layer1 = nn.Linear(784, 20)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(20, 10)
        self.layer3 = nn.Linear(784*32, 10)

        self.conv11 = nn.Conv2d(bands_nb, 32, kernel_size=3, padding=1)
        self.bn11 = nn.BatchNorm2d(32)
        self.conv12 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn12 = nn.BatchNorm2d(32)
        self.conv13 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv14 = nn.Conv2d(32, 1, kernel_size=3, padding=1)

        self.activation1 = nn.ReLU()
        self.activation2 = nn.Sigmoid()
#        self.pool = nn.MaxPool2d(2, 2)
#        self.l2norm = L2()

    def forward(self, x):
        x=F.relu(self.conv11(x)) #        x = self.pool(nn.functional.relu(self.conv1(x)))
        x=F.relu(self.conv12(x))
         # flatten all dimensions except batch
#        x = F.relu(self.layer3(x))
        x=F.relu(self.conv13(x))
        return self.conv14(x)


In [4]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bands_nb=1
patchsize=1
# load model to the specified device, either gpu or cpu
model = Classifier(bands_nb,patchsize).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()#criterion(outputs, batch_features)

In [5]:
from torchsummary import summary
summary(model,(1,28,28),batch_size=128, device='cuda')
print(model)
#print(((3*3+1)*32*31))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [128, 32, 28, 28]             320
            Conv2d-2          [128, 32, 28, 28]           9,248
            Conv2d-3          [128, 32, 28, 28]           9,248
            Conv2d-4           [128, 1, 28, 28]             289
Total params: 19,105
Trainable params: 19,105
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.38
Forward/backward pass size (MB): 74.27
Params size (MB): 0.07
Estimated Total Size (MB): 74.72
----------------------------------------------------------------
Classifier(
  (layer1): Linear(in_features=784, out_features=20, bias=True)
  (act1): ReLU()
  (layer2): Linear(in_features=20, out_features=10, bias=True)
  (layer3): Linear(in_features=25088, out_features=10, bias=True)
  (conv11): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [6]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(
    root=filedrive, train=True, transform=transform, download=True)

test_dataset = torchvision.datasets.MNIST(
    root=filedrive, train=False, transform=transform, download=True)

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [8]:
epochs = 5
from tqdm import tqdm

pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    sleep(0.25)
    pbar.set_description("Processing %s" % char)
    
pbar = tqdm(range(epochs))
for epoch in pbar:
    loss = 0
    for batch_features, batch_labels in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        #batch_features = batch_features.view(-1, 784).to(device)
        batch_features = batch_features.to(device)
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()

        # compute reconstructions
        outputs = model(batch_features)
        #outputs = outputs.view(-1).to(device)

        # compute training reconstruction loss

#        train_loss = criterion(outputs, batch_labels.to(device))
        train_loss = criterion(outputs, batch_features)

        # compute accumulated gradients
        train_loss.backward()

        # perform parameter update based on current gradients
        optimizer.step()

        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()

    # compute the epoch training loss
    loss = loss / len(train_loader)

    # display the epoch training loss
    pbar.set_description("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))


epoch : 1/5, loss = 0.002411
epoch : 2/5, loss = 0.000029
epoch : 3/5, loss = 0.000017
epoch : 4/5, loss = 0.000016
epoch : 5/5, loss = 0.000012


In [9]:
import matplotlib.pyplot as plt
import numpy as np
total = 0
correct = 0
print(test_loader.dataset)
for batch_features, labels in test_loader:
#    batch_features = batch_features.view(-1, 784).to(device)  
    batch_features = batch_features.to(device)
    outputs = model(batch_features)
    #print(outputs.shape)

    #print('IMAGE')

    #plt.imshow(batch_features[0].cpu().detach().view(28, 28).numpy())
    #plt.show()     

    #print(np.argmax(outputs[0].cpu().detach().numpy()))   

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Dataset MNIST
    Number of datapoints: 10000
    Root location: data/0Satellite_Images_Unsupervised/MNIST_PyTorch
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
           )


RuntimeError: The size of tensor a (28) must match the size of tensor b (32) at non-singleton dimension 2

In [ ]:
import matplotlib.pyplot as plt
from numpy import array
print(batch_features.shape)
#print(batch_features[0,0,:,:])
print(labels[0])
n=16
numpy_data=batch_features[0:n,0,:,:].cpu().detach().view(n,28, 28).numpy()
outputs = model(batch_features[0:n,:,:,:])
#_, predicted = torch(outputs.data, 1)
pred=outputs.cpu().detach().numpy()
#print("predicted=",predicted.cpu().detach().numpy())
print(pred.shape)
#fig, axs = plt.subplots(nrow, nrow, figsize=(20,20))

fig=plt.figure(n, figsize=(10,10))
for i in range(n):
  fig.add_subplot(1, n,  1+i) 
  plt.imshow(numpy_data[i,:,:],label="predicted="+str(pred[i])) 
#  axs[i1,i2].set_title('Th={Theta_title:.2f}  E={E_title:.3f}'.format(Theta_title=Theta,E_title=E), fontsize=12)    

plt.show()

fig=plt.figure(n, figsize=(10,10))
for i in range(n):
  fig.add_subplot(1, n,  1+i) 
  plt.imshow(pred[i,0,:,:],label="predicted="+str(pred[i])) 

#  axs[i1,i2].set_title('Th={Theta_title:.2f}  E={E_title:.3f}'.format(Theta_title=Theta,E_title=E), fontsize=12)    

plt.show()